In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from sklearn.metrics import accuracy_score

import warnings
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")

In [2]:
train_input=pd.read_csv("Credit_Risk_Train_data.csv")
test_input=pd.read_csv("Credit_Risk_Validate_data.csv")

In [3]:
print(train_input.columns)
print(test_input.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')
Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'outcome'],
      dtype='object')


In [4]:
#rename the outcome and laon status i.e., last columns
test_input.rename(columns={"outcome":"Loan_Status"},inplace=True)

In [5]:
data_all=pd.concat([train_input,test_input],axis=0)
data_all.shape

(981, 13)

In [6]:
print(data_all.tail())
data_all.reset_index(inplace=True,drop=True)

      Loan_ID Gender Married Dependents     Education Self_Employed  \
362  LP002971   Male     Yes         3+  Not Graduate           Yes   
363  LP002975   Male     Yes          0      Graduate            No   
364  LP002980   Male      No          0      Graduate            No   
365  LP002986   Male     Yes          0      Graduate            No   
366  LP002989   Male      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
362             4009             1777.0       113.0             360.0   
363             4158              709.0       115.0             360.0   
364             3250             1993.0       126.0             360.0   
365             5000             2393.0       158.0             360.0   
366             9200                0.0        98.0             180.0   

     Credit_History Property_Area Loan_Status  
362             1.0         Urban           Y  
363             1.0         Urban     

In [7]:
data_all.isnull().sum()#gives the missing values of column

Loan_ID               0
Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [8]:
Counter(data_all['Gender'])


Counter({'Male': 775, 'Female': 182, nan: 24})

In [9]:
print(data_all[data_all['Gender'].isnull()].index.tolist())
gender_null=data_all[data_all["Gender"].isnull()].index.tolist()

[23, 126, 171, 188, 314, 334, 460, 467, 477, 507, 576, 588, 592, 636, 665, 720, 752, 823, 845, 859, 893, 910, 917, 932]


In [10]:
#we can also assign 1 to 6 female and 7 onward male using slicing

data_all['Gender'].iloc[gender_null]="Male"

In [11]:
print(sum(data_all["Gender"].isnull()))#okay done
Counter(data_all['Gender'])

0


Counter({'Male': 799, 'Female': 182})

In [12]:
print(Counter(data_all["Married"]))

Counter({'Yes': 631, 'No': 347, nan: 3})


In [13]:
married_null=data_all[data_all["Married"].isnull()].index.tolist()
married_null

[104, 228, 435]

In [14]:
data_all['Married'].iloc[married_null]="Yes"

In [15]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [16]:
Counter(data_all["Dependents"])

Counter({'0': 545, '1': 160, '2': 160, '3+': 91, nan: 25})

In [17]:
#lets see the dependants vs marriage

In [18]:
pd.crosstab(data_all["Dependents"],data_all["Married"])

Married,No,Yes
Dependents,,
0,276,269
1,36,124
2,14,146
3+,12,79


In [19]:
#here the first line gives us list of missing values
bachelor_nulldependant=data_all[(data_all['Married']=='No')&(data_all["Dependents"].isnull())].index.tolist()
print(bachelor_nulldependant)

[293, 332, 355, 597, 684, 752, 879, 916, 926]


In [20]:
#if bachelor then dependancy is zero
data_all["Dependents"].iloc[bachelor_nulldependant]='0'

In [21]:
Counter(data_all["Dependents"])

Counter({'0': 554, '1': 160, '2': 160, '3+': 91, nan: 16})

In [22]:
#for remaining 16 lets see how many males anf fenmales
pd.crosstab(data_all["Gender"],data_all["Dependents"])

Dependents,0,1,2,3+
Gender,,,,
Female,127,32,13,9
Male,427,128,147,82


In [23]:
#so female have less dependants
#lets see the gender of the missing 16 dependants
data_all['Gender'].iloc[data_all[data_all['Dependents'].isnull()].index.tolist()]

102      Male
104      Male
120      Male
226      Male
228      Male
301      Male
335      Male
346      Male
435    Female
517      Male
571      Male
660      Male
725      Male
816      Male
861      Male
865      Male
Name: Gender, dtype: object

In [24]:
pd.crosstab((data_all["Gender"]=="Male")&(data_all["Married"]=='Yes'),data_all["Dependents"])

Dependents,0,1,2,3+
row_0,,,,
False,318,48,23,15
True,236,112,137,76


In [25]:
#1 is in string bcoz it is obj 
data_all["Dependents"].iloc[data_all[data_all["Dependents"].isnull()].index.tolist()]='1'

In [26]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [27]:
Counter(data_all["Self_Employed"])

Counter({'No': 807, 'Yes': 119, nan: 55})

In [28]:
self_emp_null=data_all[data_all['Self_Employed'].isnull()].index.tolist()

In [29]:
data_all['Self_Employed'].iloc[self_emp_null]="No"

In [30]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [31]:
#to check if any row with loanamount and loan amount is Nan
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'].isnull())

Loan_Amount_Term,False,True
LoanAmount,,
False,934,20
True,27,0


In [32]:
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'])

Loan_Amount_Term,6.0,12.0,36.0,60.0,84.0,120.0,180.0,240.0,300.0,350.0,360.0,480.0
LoanAmount,,,,,,,,,,,,
False,1,2,3,3,7,4,64,7,20,1,800,22
True,0,0,0,0,0,0,2,1,0,0,23,1


In [33]:
#kk=data_all.groupby(data_all['Loan_Amount_Term'])
#print(kk.first())

In [34]:
data_all.groupby(data_all['Loan_Amount_Term'])["LoanAmount"].mean()

Loan_Amount_Term
6.0       95.000000
12.0     185.500000
36.0     117.666667
60.0     139.666667
84.0     121.142857
120.0     36.750000
180.0    131.125000
240.0    128.857143
300.0    166.250000
350.0    133.000000
360.0    144.420000
480.0    137.181818
Name: LoanAmount, dtype: float64

In [35]:
#LAT having 360 and LA is true are 23 very less than 800 so assign them 144(mean)
#same for 480 
#remaining three missing values are between 180 and 240 mean of their mean value is 130
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())&(data_all['Loan_Amount_Term']==360)]=144
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())&(data_all['Loan_Amount_Term']==480)]=137

In [36]:
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())]=130

In [37]:
#lets fill laon Amount term
(data_all["Loan_Amount_Term"]).value_counts()

360.0    823
180.0     66
480.0     23
300.0     20
240.0      8
84.0       7
120.0      4
36.0       3
60.0       3
12.0       2
350.0      1
6.0        1
Name: Loan_Amount_Term, dtype: int64

In [38]:
#lets fill the loan tenure by the mode i.e, 360
data_all["Loan_Amount_Term"][data_all['Loan_Amount_Term'].isnull()]=360

In [39]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [40]:
pd.crosstab(data_all['Self_Employed'],data_all["Credit_History"])

Credit_History,0.0,1.0
Self_Employed,,
No,134,658
Yes,14,96


In [41]:
pd.crosstab(data_all['Gender'],data_all["Credit_History"])

Credit_History,0.0,1.0
Gender,,
Female,30,135
Male,118,619


In [42]:
pd.crosstab(data_all['Education'],data_all["Credit_History"])

Credit_History,0.0,1.0
Education,,
Graduate,106,596
Not Graduate,42,158


In [43]:
pd.crosstab(data_all['Married'],data_all["Credit_History"])

Credit_History,0.0,1.0
Married,,
No,56,263
Yes,92,491


In [44]:
data_all['Credit_History'][data_all["Credit_History"].isnull()]=1

In [45]:
data_all.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [46]:
#import pickle
#fh=open("data_all.pkl",'bw')
#pickle.dump(data_all,fh)
#fh.close()

In [47]:
data_all.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,144.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [48]:
data_all_new=pd.get_dummies(data_all.drop(["Loan_ID"],axis=1),
                                    drop_first=True)
data_all_new.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
0,5849,0.0,144.0,360.0,1.0,1,0,0,0,0,0,0,0,1,1
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1,1
3,2583,2358.0,120.0,360.0,1.0,1,1,0,0,0,1,0,0,1,1
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,0,0,0,1,1


In [49]:
x=data_all_new.drop(["Loan_Status_Y"],axis=1)
y=data_all_new["Loan_Status_Y"]
y.head()
x.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,144.0,360.0,1.0,1,0,0,0,0,0,0,0,1
1,4583,1508.0,128.0,360.0,1.0,1,1,1,0,0,0,0,0,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,0,0,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,1,1,0,0,0,1,0,0,1
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,0,0,0,1


In [50]:
###TRAIN TEST SPLIT########
from sklearn.model_selection  import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)
print(x_train.shape)
print(x_test.shape)


(735, 14)
(246, 14)


In [51]:
#####DATA PROCCESSING#####
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
#fit only to the training data
scaler.fit(x)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [52]:
##now apply the transformations to the data
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [59]:
#####TRAINING THE MODEL#####
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(14,14),max_iter=500)
mlp.fit(x_train,y_train)
predictions=mlp.predict(x_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))


[[ 41  34]
 [ 10 161]]
              precision    recall  f1-score   support

           0       0.80      0.55      0.65        75
           1       0.83      0.94      0.88       171

   micro avg       0.82      0.82      0.82       246
   macro avg       0.81      0.74      0.77       246
weighted avg       0.82      0.82      0.81       246



In [53]:
model = LogisticRegression()

In [55]:
model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [58]:
predict = model.predict(x_test)

print('Predicted Values on Test Data',predict)

print('\n\nAccuracy Score on test data : \n\n')
print(accuracy_score(y_test,predict))

Predicted Values on Test Data [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1
 1 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1]


Accuracy Score on test data : 


0.8292682926829268
